### Modules importeren

In [3]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

import os
from loguru import logger # pip install loguru
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

servername = 'NOAH' # PAS DIT AAN NAAR JE EIGEN SERVER!!!

# Data inlezen

### Connecties maken

In [4]:
# SQLite Connecties
logger.info('Establishing connection with SQLite databases...')
northwind_sqlite_conn = sqlite3.connect('../data/processed/northwind.sqlite')
adventureworks_sqlite_conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
aenc_sqlite_conn = sqlite3.connect('../data/processed/aenc.sqlite')
logger.success('Connections established!')

# Connect to the SQLite database
conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
cursor = conn.cursor()


2024-05-23 11:28:34.613 | INFO     | __main__:<module>:2 - Establishing connection with SQLite databases...
2024-05-23 11:28:34.614 | SUCCESS  | __main__:<module>:6 - Connections established!


# Dataframes maken

### Person

In [5]:
#Collecting tables
person = pd.read_sql('SELECT * FROM "Person.Person"', adventureworks_sqlite_conn)
businessEntity = pd.read_sql('SELECT * FROM "Person.BusinessEntity"', adventureworks_sqlite_conn)
password = pd.read_sql('SELECT * FROM "Person.Password"', adventureworks_sqlite_conn)
emailAddress = pd.read_sql('SELECT * FROM "Person.EmailAddress"', adventureworks_sqlite_conn)
personPhone = pd.read_sql('SELECT * FROM "Person.PersonPhone"', adventureworks_sqlite_conn)
phoneNumberType = pd.read_sql('SELECT * FROM "Person.PhoneNumberType"', adventureworks_sqlite_conn)
businessEntityContact = pd.read_sql('SELECT * FROM "Person.BusinessEntityContact"', adventureworks_sqlite_conn)
contactType = pd.read_sql('SELECT * FROM "Person.ContactType"', adventureworks_sqlite_conn)
vendor = pd.read_sql('SELECT * FROM "Purchasing.Vendor"', adventureworks_sqlite_conn)

#Merges
person = pd.merge(person, businessEntity, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntity'))
person = pd.merge(person, password, on=['BusinessEntityID'], how='outer', suffixes=('', '_password'))
person = pd.merge(person, emailAddress, on=['BusinessEntityID'], how='outer', suffixes=('', '_emailAddress'))
person = pd.merge(person, personPhone, on=['BusinessEntityID'], how='outer', suffixes=('', '_personPhone'))
person = pd.merge(person, phoneNumberType, on=['PhoneNumberTypeID'], how='outer', suffixes=('', '_phoneNumberType'))
person = pd.merge(person, businessEntityContact, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntityContact'))
person = pd.merge(person, contactType, on=['ContactTypeID'], how='outer', suffixes=('', '_contactType'))
person = pd.merge(person, vendor, on=['BusinessEntityID'], how='outer', suffixes=('', '_vendor'))

#Get most recent ModifiedDate
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntity'] if pd.to_datetime(row['ModifiedDate_businessEntity']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_password'] if pd.to_datetime(row['ModifiedDate_password']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_emailAddress'] if pd.to_datetime(row['ModifiedDate_emailAddress']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_personPhone'] if pd.to_datetime(row['ModifiedDate_personPhone']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_phoneNumberType'] if pd.to_datetime(row['ModifiedDate_phoneNumberType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntityContact'] if pd.to_datetime(row['ModifiedDate_businessEntityContact']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_contactType'] if pd.to_datetime(row['ModifiedDate_contactType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_vendor'] if pd.to_datetime(row['ModifiedDate_vendor']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
person.drop(columns=['ModifiedDate_businessEntity'], inplace=True)
person.drop(columns=['ModifiedDate_password'], inplace=True)
person.drop(columns=['ModifiedDate_emailAddress'], inplace=True)
person.drop(columns=['ModifiedDate_personPhone'], inplace=True)
person.drop(columns=['ModifiedDate_phoneNumberType'], inplace=True)
person.drop(columns=['ModifiedDate_businessEntityContact'], inplace=True)
person.drop(columns=['ModifiedDate_contactType'], inplace=True)
person.drop(columns=['ModifiedDate_vendor'], inplace=True)

person

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,...,PersonID,ContactTypeID,rowguid_businessEntityContact,Name_contactType,AccountNumber,Name_vendor,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL
0,1.0,EM,0.0,None,Ken,J,Sánchez,None,0.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,EM,0.0,None,Terri,Lee,Duffy,None,1.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,EM,0.0,None,Roberto,None,Tamburello,None,0.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,EM,0.0,None,Rob,None,Walters,None,0.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,EM,0.0,Ms.,Gail,A,Erickson,None,0.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,Order Administrator,NaN,NaN,NaN,NaN,NaN,NaN
20890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12.0,NaN,Owner/Marketing Assistant,NaN,NaN,NaN,NaN,NaN,NaN
20891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,Product Manager,NaN,NaN,NaN,NaN,NaN,NaN
20892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,16.0,NaN,Regional Account Representative,NaN,NaN,NaN,NaN,NaN,NaN


### Product

In [6]:
#Collecting tables
product_adventureworks = pd.read_sql('SELECT * FROM "Production.Product"', adventureworks_sqlite_conn)
productModel = pd.read_sql('SELECT * FROM "Production.ProductModel"', adventureworks_sqlite_conn)
productSubcategory = pd.read_sql('SELECT * FROM "Production.ProductSubcategory"', adventureworks_sqlite_conn)
productCategory = pd.read_sql('SELECT * FROM "Production.ProductCategory"', adventureworks_sqlite_conn)

product_northwind = pd.read_sql('SELECT * FROM "Products"', northwind_sqlite_conn)
categories = pd.read_sql('SELECT * FROM "Categories"', northwind_sqlite_conn)
suppliers = pd.read_sql('SELECT * FROM "Suppliers"', northwind_sqlite_conn)

product_aenc = pd.read_sql('SELECT * FROM "product"', aenc_sqlite_conn)

#Merges
product_adventureworks = pd.merge(product_adventureworks, productModel, on=['ProductModelID'], how='outer', suffixes=('', '_productModel'))
product_adventureworks = pd.merge(product_adventureworks, productSubcategory, on=['ProductSubcategoryID'], how='outer', suffixes=('', '_productSubcategory'))
product_adventureworks = pd.merge(product_adventureworks, productCategory, on=['ProductCategoryID'], how='outer', suffixes=('', '_productCategory'))

product_northwind = pd.merge(product_northwind, categories, on=['CategoryID'], how='outer', suffixes=('', '_Category'))
product_northwind = pd.merge(product_northwind, suppliers, on=['SupplierID'], how='outer', suffixes=('', '_Supplier'))

product = pd.concat([product_adventureworks, product_northwind, product_aenc])

#Merging Mergable Columns
product['Name'] = product['Name'].combine_first(product['name']).combine_first(product['ProductName'])
product.drop(columns=['ProductName', 'name'], inplace=True)

product['CategoryName'] = product['Category'].combine_first(product['CategoryName']).combine_first(product['Name_productCategory'])
product.drop(columns=['Name_productCategory', 'Category', 'CategoryName'], inplace=True)


#Get most recent ModifiedDate
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productModel'] if pd.to_datetime(row['ModifiedDate_productModel']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productCategory'] if pd.to_datetime(row['ModifiedDate_productCategory']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = product.apply(lambda row: row['ModifiedDate_productSubcategory'] if pd.to_datetime(row['ModifiedDate_productSubcategory']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
product.drop(columns=['ModifiedDate_productModel'], inplace=True)
product.drop(columns=['ModifiedDate_productCategory'], inplace=True)
product.drop(columns=['ModifiedDate_productSubcategory'], inplace=True)

product

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Phone,Fax,HomePage,id,description,prod_size,color,quantity,unit_price,picture_name
0,771.0,"Mountain-100 Silver, 38",BK-M82S-38,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,772.0,"Mountain-100 Silver, 42",BK-M82S-42,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,773.0,"Mountain-100 Silver, 44",BK-M82S-44,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,774.0,"Mountain-100 Silver, 48",BK-M82S-48,1.0,1.0,Silver,100.0,75.0,1912.1544,3399.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,775.0,"Mountain-100 Black, 38",BK-M82B-38,1.0,1.0,Black,100.0,75.0,1898.0944,3374.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,NaN,Visor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,500,Cloth Visor,One size fits all,White,36,7,visorw.bmp
6,NaN,Visor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,501,Plastic Visor,One size fits all,Black,28,7,visorb.bmp
7,NaN,Sweatshirt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,600,Hooded Sweatshirt,Large,Green,39,24,sshirtg.bmp
8,NaN,Sweatshirt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,601,Zipped Sweatshirt,Large,Blue,32,24,sshirtb.bmp


### Employees

In [7]:
#Collecting tables
employee_adventureworks = pd.read_sql('SELECT * FROM "HumanResources.Employee"', adventureworks_sqlite_conn)
department_adventureworks = pd.read_sql('SELECT * FROM "HumanResources.Department"', adventureworks_sqlite_conn)

employee_northwind = pd.read_sql('SELECT * FROM "Employees"', northwind_sqlite_conn)
employee_territories = pd.read_sql('SELECT * FROM "EmployeeTerritories"', northwind_sqlite_conn)

employee_aenc = pd.read_sql('SELECT * FROM "employee"', aenc_sqlite_conn)
department_aenc = pd.read_sql('SELECT * FROM "department"', aenc_sqlite_conn)

#Merges
department = pd.concat([department_adventureworks, department_aenc])

employee_northwind = pd.merge(employee_northwind, employee_territories, on=['EmployeeID'], how='outer', suffixes=('', '_employee_territories'))

employee_aenc = pd.merge(employee_aenc, department, on=['dept_id'], how='outer', suffixes=('', '_department'))

employee = pd.concat([employee_adventureworks, employee_northwind, employee_aenc])

#Merging Mergable Columns
employee['dept_name'] = employee['dept_name'].combine_first(employee['Name'])
employee.drop(columns=['Name'], inplace=True)

employee

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,...,termination_date,birth_date,bene_health_ins,bene_life_ins,bene_day_care,sex,DepartmentID,GroupName,dept_name,dept_head_id
0,1.0,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,245797967,adventure-works\terri0,b'X',1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,509647174,adventure-works\roberto0,b'Z\xc0',2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,112457891,adventure-works\rob0,b'Z\xd6',3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,695256908,adventure-works\gail0,b'Z\xda',3.0,Design Engineer,1952-09-27,M,F,2008-01-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,Quality Assurance,Document Control,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Quality Assurance,Quality Assurance,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,Executive General and Administration,Facilities and Maintenance,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,Inventory Management,Shipping and Receiving,NaN


### Customer

In [8]:
#Collecting tables
customer_adventureworks = pd.read_sql('SELECT * FROM "Sales.Customer"', adventureworks_sqlite_conn)

customer_northwind = pd.read_sql('SELECT * FROM "Customers"', northwind_sqlite_conn)

customer_aenc = pd.read_sql('SELECT * FROM "customer"', aenc_sqlite_conn)

#Merges
customer = pd.concat([customer_adventureworks, customer_northwind, customer_aenc])

customer

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate,CompanyName,ContactName,ContactTitle,...,Fax,id,fname,lname,address,city,state,zip,phone,company_name
0,1,NaN,934.0,1.0,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1028.0,1.0,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,642.0,4.0,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,932.0,4.0,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,1026.0,4.0,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


### StateProvince

In [9]:
#Collecting tables
stateProvince_adventureworks = pd.read_sql('SELECT * FROM "Person.StateProvince"', adventureworks_sqlite_conn)
CountryRegion = pd.read_sql('SELECT * FROM "Person.CountryRegion"', adventureworks_sqlite_conn)

stateProvince_northwind = pd.read_sql('SELECT * FROM "Region"', northwind_sqlite_conn)

stateProvince_aenc = pd.read_sql('SELECT * FROM "state"', aenc_sqlite_conn)
region = pd.read_sql('SELECT * FROM "region"', aenc_sqlite_conn)

#Merges
stateProvince_adventureworks = pd.merge(stateProvince_adventureworks, CountryRegion, on=['CountryRegionCode'], how='outer', suffixes=('', '_countryRegion'))
stateProvince_aenc = pd.merge(stateProvince_aenc, region, on=['region'], how='outer', suffixes=('', '_region'))

stateProvince = pd.concat([stateProvince_adventureworks, stateProvince_northwind, stateProvince_aenc])

#Merging Mergable Columns
stateProvince['Name'] = stateProvince['Name'].combine_first(stateProvince['state_name'])
stateProvince.drop(columns=['state_name'], inplace=True)

#Get most recent ModifiedDate
stateProvince['ModifiedDate'] = stateProvince.apply(lambda row: row['ModifiedDate_countryRegion'] if pd.to_datetime(row['ModifiedDate_countryRegion']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
stateProvince.drop(columns=['ModifiedDate_countryRegion'], inplace=True)

stateProvince

,StateProvinceID,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,Name,TerritoryID,rowguid,ModifiedDate,Name_countryRegion,RegionID,RegionDescription,state_id,state_capital,country,region
0,NaN,NaN,AD,NaN,NaN,NaN,NaN,NaN,Andorra,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,AE,NaN,NaN,NaN,NaN,NaN,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,AF,NaN,NaN,NaN,NaN,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,AG,NaN,NaN,NaN,NaN,NaN,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,AI,NaN,NaN,NaN,NaN,NaN,Anguilla,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,Oregon,NaN,NaN,NaN,NaN,NaN,NaN,OR,Salem,USA,Western
60,NaN,NaN,NaN,NaN,Utah,NaN,NaN,NaN,NaN,NaN,NaN,UT,Salt Lake City,USA,Western
61,NaN,NaN,NaN,NaN,Washington,NaN,NaN,NaN,NaN,NaN,NaN,WA,Olympia,USA,Western
62,NaN,NaN,NaN,NaN,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,WY,Cheyenne,USA,Western


### Address

In [10]:
#Collecting tables
address = pd.read_sql('SELECT * FROM "Person.Address"', adventureworks_sqlite_conn)
addressType = pd.read_sql('SELECT * FROM "Person.AddressType"', adventureworks_sqlite_conn)
businessEntityAddress = pd.read_sql('SELECT * FROM "Person.BusinessEntityAddress"', adventureworks_sqlite_conn)

#Merges
address = pd.merge(address, businessEntityAddress, on=['AddressID'], how='outer', suffixes=('', '_businessEntityAddress'))
address = pd.merge(address, addressType, on=['AddressTypeID'], how='outer', suffixes=('', '_addressType'))

#Get most recent ModifiedDate
address['ModifiedDate'] = address.apply(lambda row: row['ModifiedDate_addressType'] if pd.to_datetime(row['ModifiedDate_addressType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
address['ModifiedDate'] = address.apply(lambda row: row['ModifiedDate_businessEntityAddress'] if pd.to_datetime(row['ModifiedDate_businessEntityAddress']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
address.drop(columns=['ModifiedDate_addressType'], inplace=True)
address.drop(columns=['ModifiedDate_businessEntityAddress'], inplace=True)

address

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,rowguid,ModifiedDate,BusinessEntityID,AddressTypeID,rowguid_businessEntityAddress,Name,rowguid_addressType
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Billing,B84F78B1-4EFE-4A0E-8CB7-70E9F112F886
1,1.0,1970 Napa Ct.,None,Bothell,79.0,98011,b'\xe6\x10\x00\x00\x01\x0c\xae\x8b\xfc(\xbc\xe...,9AADCB0D-36CF-483F-84D8-585C2D4EC6E9,2014-09-12 11:15:06.967000,12.0,2,8AA698FC-090F-42EE-A197-2E7F7394D9F1,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2
2,2.0,9833 Mt. Dias Blv.,None,Bothell,79.0,98011,b'\xe6\x10\x00\x00\x01\x0c\xd6\xfa\x85\x1a\xe6...,32A54B9E-E034-4BFB-B573-A71CDE60D8C0,2014-09-12 11:15:06.967000,123.0,2,E5C2EA88-0499-45C4-9C27-938E1DBF9C37,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2
3,3.0,7484 Roundtree Drive,None,Bothell,79.0,98011,b'\xe6\x10\x00\x00\x01\x0c\x18\xe3\x04\xc4\xad...,4C506923-6D1B-452C-A07C-BAA6F5B142A4,2014-09-12 11:15:06.967000,285.0,2,DACFBC8F-1F78-4909-A25D-232034A831C1,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2
4,4.0,9539 Glenside Dr,None,Bothell,79.0,98011,b'\xe6\x10\x00\x00\x01\x0c\x81:\r_\x9f\xdeG@\x...,E5946C78-4BCC-477F-9FA1-CC09DE16A880,2014-09-12 11:15:06.967000,251.0,2,1FCE9144-06CB-467D-AA5D-B2EA6F68D019,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19612,11382.0,99 Front Street,None,Minneapolis,36.0,55402,b'\xe6\x10\x00\x00\x01\x0c& Y\xc7\nzF@^M\xfa\x...,878660A8-A9FE-4EF8-8D2D-53C614C6F042,2011-07-01 00:00:00,1126.0,5,19FCC0C7-CE20-4648-BD54-FF8A77ADCF3D,Shipping,B29DA3F8-19A3-47DA-9DAA-15C84F4A83A5
19613,11383.0,1010 Maple,None,Baltimore,32.0,21201,b'\xe6\x10\x00\x00\x01\x0cY\xfa\x01\x9d\xc4\xa...,6D3E3068-8792-4567-8D57-018B062D924B,2014-04-06 00:00:00,8892.0,5,C5229769-8902-42BC-96F0-BF4C794D2231,Shipping,B29DA3F8-19A3-47DA-9DAA-15C84F4A83A5
19614,11384.0,500 35th Ave NE,None,Los Angeles,9.0,90012,b'\xe6\x10\x00\x00\x01\x0c\xc3\xf3\xc8x \rA@\t...,35DF6920-D4E9-47CB-9851-6E0AA1B4ED6E,2013-03-14 00:00:00,5479.0,5,DB050410-F033-42AC-BA53-44A40A6295BB,Shipping,B29DA3F8-19A3-47DA-9DAA-15C84F4A83A5
19615,11385.0,9 Olive Way,None,Seattle,79.0,98104,b'\xe6\x10\x00\x00\x01\x0c\xa5\x0b\xceM\x94\xc...,ABBE790A-4971-498B-94F0-A3DB85E22984,2013-11-13 00:00:00,16746.0,5,BE430A32-C446-44F0-93D7-4EFCAF90DB9F,Shipping,B29DA3F8-19A3-47DA-9DAA-15C84F4A83A5


### WorkOrderRouting

In [11]:
#Collecting tables
workOrderRouting = pd.read_sql('SELECT * FROM "Production.WorkOrderRouting"', adventureworks_sqlite_conn)

workOrderRouting

,WorkOrderID,ProductID,OperationSequence,LocationID,ScheduledStartDate,ScheduledEndDate,ActualStartDate,ActualEndDate,ActualResourceHrs,PlannedCost,ActualCost,ModifiedDate
0,13,747,1,10,2011-06-03 00:00:00,2011-06-14 00:00:00,2011-06-03 00:00:00,2011-06-19 00:00:00,4.1,92.25,92.25,2011-06-19 00:00:00
1,13,747,2,20,2011-06-03 00:00:00,2011-06-14 00:00:00,2011-06-03 00:00:00,2011-06-19 00:00:00,3.5,87.50,87.50,2011-06-19 00:00:00
2,13,747,3,30,2011-06-03 00:00:00,2011-06-14 00:00:00,2011-06-03 00:00:00,2011-06-19 00:00:00,1.0,14.50,14.50,2011-06-19 00:00:00
3,13,747,4,40,2011-06-03 00:00:00,2011-06-14 00:00:00,2011-06-03 00:00:00,2011-06-19 00:00:00,2.0,31.50,31.50,2011-06-19 00:00:00
4,13,747,6,50,2011-06-03 00:00:00,2011-06-14 00:00:00,2011-06-03 00:00:00,2011-06-19 00:00:00,3.0,36.75,36.75,2011-06-19 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
67126,72585,802,6,50,2014-06-02 00:00:00,2014-06-13 00:00:00,2014-06-05 00:00:00,2014-06-13 00:00:00,3.0,36.75,36.75,2014-06-13 00:00:00
67127,72586,803,1,10,2014-06-02 00:00:00,2014-06-13 00:00:00,2014-06-05 00:00:00,2014-06-13 00:00:00,4.1,92.25,92.25,2014-06-13 00:00:00
67128,72586,803,6,50,2014-06-02 00:00:00,2014-06-13 00:00:00,2014-06-05 00:00:00,2014-06-13 00:00:00,3.0,36.75,36.75,2014-06-13 00:00:00
67129,72587,804,1,10,2014-06-02 00:00:00,2014-06-13 00:00:00,2014-06-05 00:00:00,2014-06-13 00:00:00,4.1,92.25,92.25,2014-06-13 00:00:00


### WorkOrderRouting

In [12]:
#Collecting tables
workOrder = pd.read_sql('SELECT * FROM "Production.WorkOrder"', adventureworks_sqlite_conn)

workOrder

,WorkOrderID,ProductID,OrderQty,StockedQty,ScrappedQty,StartDate,EndDate,DueDate,ScrapReasonID,ModifiedDate
0,1,722,8,8,0,2011-06-03 00:00:00,2011-06-13 00:00:00,2011-06-14 00:00:00,NaN,2011-06-13 00:00:00
1,2,725,15,15,0,2011-06-03 00:00:00,2011-06-13 00:00:00,2011-06-14 00:00:00,NaN,2011-06-13 00:00:00
2,3,726,9,9,0,2011-06-03 00:00:00,2011-06-13 00:00:00,2011-06-14 00:00:00,NaN,2011-06-13 00:00:00
3,4,729,16,16,0,2011-06-03 00:00:00,2011-06-13 00:00:00,2011-06-14 00:00:00,NaN,2011-06-13 00:00:00
4,5,730,14,14,0,2011-06-03 00:00:00,2011-06-13 00:00:00,2011-06-14 00:00:00,NaN,2011-06-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...
72586,72587,804,19,19,0,2014-06-02 00:00:00,2014-06-13 00:00:00,2014-06-13 00:00:00,NaN,2014-06-13 00:00:00
72587,72588,316,52,52,0,2014-06-02 00:00:00,2014-06-12 00:00:00,2014-06-13 00:00:00,NaN,2014-06-12 00:00:00
72588,72589,331,52,52,0,2014-06-02 00:00:00,2014-06-12 00:00:00,2014-06-13 00:00:00,NaN,2014-06-12 00:00:00
72589,72590,350,26,26,0,2014-06-02 00:00:00,2014-06-12 00:00:00,2014-06-13 00:00:00,NaN,2014-06-12 00:00:00


### Document

In [13]:
#Collecting tables
document = pd.read_sql('SELECT * FROM "Production.Document"', adventureworks_sqlite_conn)
productDocument = pd.read_sql('SELECT * FROM "Production.ProductDocument"', adventureworks_sqlite_conn)

#Merges
document = pd.merge(document, productDocument, on=['DocumentNode'], how='outer', suffixes=('', '_productDocument'))

document

,DocumentNode,DocumentLevel,Title,Owner,FolderFlag,FileName,FileExtension,Revision,ChangeNumber,Status,DocumentSummary,Document,rowguid,ModifiedDate,ProductID,ModifiedDate_productDocument
0,b'',0,Documents,217,1,Documents,,0,0,2,None,None,27CF33AF-C338-4842-966C-75CA11AAA6A3,2017-12-13 13:58:03.943000,NaN,NaN
1,b'X',1,Overview,217,1,Overview,,0,0,2,None,None,26A266F1-1D23-40E2-AF48-6AB8D954FE37,2017-12-13 13:58:03.977000,NaN,NaN
2,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,977.0,2013-12-29 13:51:58.103000
3,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,997.0,2013-12-29 13:51:58.103000
4,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,998.0,2013-12-29 13:51:58.103000
5,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,999.0,2013-12-29 13:51:58.120000
6,b'[@',2,Repair and Service Guidelines,220,0,Repair and Service Guidelines.doc,.doc,0,8,2,It is important that you maintain your bicycle...,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,7ED4DEF5-D5BB-4818-8748-5BB5F8315FA2,2008-03-31 00:00:00,NaN,NaN
7,b'h',1,Maintenance,217,1,Maintenance,,0,0,2,None,None,5184D96A-EE8C-499A-9316-625496784DE6,2017-12-13 13:58:04.007000,NaN,NaN
8,b'j\xc0',2,Crank Arm and Tire Maintenance,219,0,Crank Arm and Tire Maintenance.doc,.doc,0,0,1,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,D11B82B0-EB40-4A66-8193-15F765A61F05,2012-10-19 09:56:38.273000,317.0,2013-12-29 13:51:58.103000
9,b'j\xc0',2,Crank Arm and Tire Maintenance,219,0,Crank Arm and Tire Maintenance.doc,.doc,0,0,1,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,D11B82B0-EB40-4A66-8193-15F765A61F05,2012-10-19 09:56:38.273000,318.0,2013-12-29 13:51:58.103000


### Document

In [14]:
#Collecting tables
document = pd.read_sql('SELECT * FROM "Production.Document"', adventureworks_sqlite_conn)
productDocument = pd.read_sql('SELECT * FROM "Production.ProductDocument"', adventureworks_sqlite_conn)

#Merges
document = pd.merge(document, productDocument, on=['DocumentNode'], how='outer', suffixes=('', '_productDocument'))

document

,DocumentNode,DocumentLevel,Title,Owner,FolderFlag,FileName,FileExtension,Revision,ChangeNumber,Status,DocumentSummary,Document,rowguid,ModifiedDate,ProductID,ModifiedDate_productDocument
0,b'',0,Documents,217,1,Documents,,0,0,2,None,None,27CF33AF-C338-4842-966C-75CA11AAA6A3,2017-12-13 13:58:03.943000,NaN,NaN
1,b'X',1,Overview,217,1,Overview,,0,0,2,None,None,26A266F1-1D23-40E2-AF48-6AB8D954FE37,2017-12-13 13:58:03.977000,NaN,NaN
2,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,977.0,2013-12-29 13:51:58.103000
3,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,997.0,2013-12-29 13:51:58.103000
4,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,998.0,2013-12-29 13:51:58.103000
5,b'Z\xc0',2,Introduction 1,219,0,Introduction 1.doc,.doc,4,28,2,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,48265174-8451-4967-973A-639C2276CFAF,2013-05-30 00:00:00,999.0,2013-12-29 13:51:58.120000
6,b'[@',2,Repair and Service Guidelines,220,0,Repair and Service Guidelines.doc,.doc,0,8,2,It is important that you maintain your bicycle...,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,7ED4DEF5-D5BB-4818-8748-5BB5F8315FA2,2008-03-31 00:00:00,NaN,NaN
7,b'h',1,Maintenance,217,1,Maintenance,,0,0,2,None,None,5184D96A-EE8C-499A-9316-625496784DE6,2017-12-13 13:58:04.007000,NaN,NaN
8,b'j\xc0',2,Crank Arm and Tire Maintenance,219,0,Crank Arm and Tire Maintenance.doc,.doc,0,0,1,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,D11B82B0-EB40-4A66-8193-15F765A61F05,2012-10-19 09:56:38.273000,317.0,2013-12-29 13:51:58.103000
9,b'j\xc0',2,Crank Arm and Tire Maintenance,219,0,Crank Arm and Tire Maintenance.doc,.doc,0,0,1,None,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,D11B82B0-EB40-4A66-8193-15F765A61F05,2012-10-19 09:56:38.273000,318.0,2013-12-29 13:51:58.103000


### SalesOrderItem

In [15]:
#Collecting tables
sales_order_item = pd.read_sql('SELECT * FROM "sales_order_item"', aenc_sqlite_conn)

sales_order_item

,id,line_id,prod_id,quantity,ship_date
0,2001,1,300,12,15-Sep-1996 12:00:00 AM
1,2001,2,301,12,14-Sep-1996 12:00:00 AM
2,2001,3,302,12,14-Sep-1996 12:00:00 AM
3,2002,1,400,24,18-Sep-1996 12:00:00 AM
4,2002,2,401,24,18-Sep-1996 12:00:00 AM
...,...,...,...,...,...
1098,2652,2,300,10,01-Dec-1998 12:00:00 AM
1099,2653,1,301,0,17-Oct-1999 12:00:00 AM
1100,2654,1,301,15,29-Oct-1999 12:00:00 AM
1101,2654,2,302,3,29-Oct-1999 12:00:00 AM


### Bonus

In [16]:
#Collecting tables
bonus = pd.read_sql('SELECT * FROM "bonus"', aenc_sqlite_conn)

bonus

,emp_id,bonus_date,bonus_amount
0,129,31-Dec-1998 12:00:00 AM,50
1,195,31-Dec-1997 12:00:00 AM,50
2,299,31-Dec-1997 12:00:00 AM,200
3,1142,31-Dec-1997 12:00:00 AM,100
4,129,31-Dec-1996 12:00:00 AM,50
5,129,31-Dec-1997 12:00:00 AM,250
6,129,31-Dec-1998 12:00:00 AM,400
7,195,31-Dec-1996 12:00:00 AM,0
8,195,31-Dec-1997 12:00:00 AM,450
9,195,31-Dec-1998 12:00:00 AM,100


### PurchaseOrderDetails

In [17]:
#Collecting tables
purchaseOrderDetails = pd.read_sql('SELECT * FROM "Purchasing.PurchaseOrderDetail"', adventureworks_sqlite_conn)
purchaseOrderHeader = pd.read_sql('SELECT * FROM "Purchasing.PurchaseOrderHeader"', adventureworks_sqlite_conn)

#Merges
purchaseOrderDetails = pd.merge(purchaseOrderDetails, purchaseOrderHeader, on=['PurchaseOrderID'], how='outer', suffixes=('', '_purchaseOrderHeader'))

#Get most recent ModifiedDate
purchaseOrderDetails['ModifiedDate'] = purchaseOrderDetails.apply(lambda row: row['ModifiedDate_purchaseOrderHeader'] if pd.to_datetime(row['ModifiedDate_purchaseOrderHeader']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
purchaseOrderDetails.drop(columns=['ModifiedDate_purchaseOrderHeader'], inplace=True)

purchaseOrderDetails

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,...,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue
0,1,1,2011-04-30 00:00:00,4,1,50.2600,201.0400,3.0,0.0,3.0,...,4,258,1580,3,2011-04-16 00:00:00,2011-04-25 00:00:00,201.0400,16.0832,5.0260,2.221492e+02
1,2,2,2011-04-30 00:00:00,3,359,45.1200,135.3600,3.0,0.0,3.0,...,1,254,1496,5,2011-04-16 00:00:00,2011-04-25 00:00:00,272.1015,21.7681,6.8025,3.006721e+02
2,2,3,2011-04-30 00:00:00,3,360,45.5805,136.7415,3.0,0.0,3.0,...,1,254,1496,5,2011-04-16 00:00:00,2011-04-25 00:00:00,272.1015,21.7681,6.8025,3.006721e+02
3,3,4,2011-04-30 00:00:00,550,530,16.0860,8847.3000,550.0,0.0,550.0,...,4,257,1494,2,2011-04-16 00:00:00,2011-04-25 00:00:00,8847.3000,707.7840,221.1825,9.776266e+03
4,4,5,2011-04-30 00:00:00,3,4,57.0255,171.0765,2.0,1.0,1.0,...,3,261,1650,5,2011-04-16 00:00:00,2011-04-25 00:00:00,171.0765,13.6861,4.2769,1.890395e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8840,4011,8841,2014-07-24 00:00:00,1000,880,20.5600,20560.0000,1000.0,0.0,1000.0,...,2,254,1546,3,2014-06-24 00:00:00,2014-07-19 00:00:00,54492.5000,4359.4000,1089.8500,5.994175e+04
8841,4012,8842,2014-07-24 00:00:00,6000,881,41.5700,249420.0000,6000.0,0.0,6000.0,...,2,254,1636,3,2014-06-24 00:00:00,2014-07-19 00:00:00,997680.0000,79814.4000,19953.6000,1.097448e+06
8842,4012,8843,2014-07-24 00:00:00,6000,882,41.5700,249420.0000,6000.0,0.0,6000.0,...,2,254,1636,3,2014-06-24 00:00:00,2014-07-19 00:00:00,997680.0000,79814.4000,19953.6000,1.097448e+06
8843,4012,8844,2014-07-24 00:00:00,6000,883,41.5700,249420.0000,6000.0,0.0,6000.0,...,2,254,1636,3,2014-06-24 00:00:00,2014-07-19 00:00:00,997680.0000,79814.4000,19953.6000,1.097448e+06


### ProductVendor

In [18]:
#Collecting tables
productVendor = pd.read_sql('SELECT * FROM "Purchasing.ProductVendor"', adventureworks_sqlite_conn)

productVendor

,ProductID,BusinessEntityID,AverageLeadTime,StandardPrice,LastReceiptCost,LastReceiptDate,MinOrderQty,MaxOrderQty,OnOrderQty,UnitMeasureCode,ModifiedDate
0,1,1580,17,47.87,50.2635,2011-08-29 00:00:00,1,5,3.0,CS,2011-08-29 00:00:00
1,2,1688,19,39.92,41.9160,2011-08-29 00:00:00,1,5,3.0,CTN,2011-08-29 00:00:00
2,4,1650,17,54.31,57.0255,2011-08-29 00:00:00,1,5,NaN,CTN,2011-08-29 00:00:00
3,317,1578,19,28.17,29.5785,2011-08-29 00:00:00,100,1000,300.0,EA,2011-08-29 00:00:00
4,317,1678,17,25.77,27.0585,2011-08-25 00:00:00,100,1000,NaN,EA,2011-08-25 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
455,939,1680,16,45.99,48.2895,2011-08-27 00:00:00,100,1000,NaN,EA,2011-08-27 00:00:00
456,940,1508,15,59.99,62.9895,2011-08-26 00:00:00,100,1000,NaN,EA,2011-08-26 00:00:00
457,941,1628,17,59.99,62.9895,2011-08-25 00:00:00,100,1000,NaN,EA,2011-08-25 00:00:00
458,948,1576,19,78.89,82.8345,2011-08-29 00:00:00,100,1000,NaN,EA,2011-08-29 00:00:00


### BillOfMaterials

In [19]:
#Collecting tables
billOfMaterials = pd.read_sql('SELECT * FROM "Production.BillOfMaterials"', adventureworks_sqlite_conn)

billOfMaterials

,BillOfMaterialsID,ProductAssemblyID,ComponentID,StartDate,EndDate,UnitMeasureCode,BOMLevel,PerAssemblyQty,ModifiedDate
0,893,NaN,749,2010-05-26 00:00:00,None,EA,0,1.0,2010-05-12 00:00:00
1,271,NaN,750,2010-03-04 00:00:00,2010-05-03 00:00:00,EA,0,1.0,2010-05-03 00:00:00
2,34,NaN,750,2010-05-04 00:00:00,None,EA,0,1.0,2010-04-20 00:00:00
3,830,NaN,751,2010-05-26 00:00:00,None,EA,0,1.0,2010-05-12 00:00:00
4,2074,NaN,752,2010-07-08 00:00:00,None,EA,0,1.0,2010-06-24 00:00:00
...,...,...,...,...,...,...,...,...,...
2674,3063,999.0,948,2010-12-15 00:00:00,None,EA,1,1.0,2010-12-01 00:00:00
2675,75,999.0,949,2010-03-04 00:00:00,None,EA,1,1.0,2010-02-18 00:00:00
2676,285,999.0,952,2010-03-04 00:00:00,2010-05-03 00:00:00,EA,1,2.0,2010-05-03 00:00:00
2677,74,999.0,952,2010-05-04 00:00:00,None,EA,1,1.0,2010-04-20 00:00:00


### ProductListPriceHistory

In [20]:
#Collecting tables
productListPriceHistory = pd.read_sql('SELECT * FROM "Production.ProductListPriceHistory"', adventureworks_sqlite_conn)
productCostHistory = pd.read_sql('SELECT * FROM "Production.ProductListPriceHistory"', adventureworks_sqlite_conn)

#Merges
productListPriceHistory = pd.merge(productListPriceHistory, productCostHistory, on=['ProductID', 'StartDate', 'EndDate'], how='outer', suffixes=('', '_productCostHistory'))

#Get most recent ModifiedDate
productListPriceHistory['ModifiedDate'] = productListPriceHistory.apply(lambda row: row['ModifiedDate_productCostHistory'] if pd.to_datetime(row['ModifiedDate_productCostHistory']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
productListPriceHistory.drop(columns=['ModifiedDate_productCostHistory'], inplace=True)

productListPriceHistory

,ProductID,StartDate,EndDate,ListPrice,ModifiedDate,ListPrice_productCostHistory
0,707,2011-05-31 00:00:00,2012-05-29 00:00:00,33.6442,2012-05-29 00:00:00,33.6442
1,707,2012-05-30 00:00:00,2013-05-29 00:00:00,33.6442,2013-05-29 00:00:00,33.6442
2,707,2013-05-30 00:00:00,None,34.9900,2013-05-09 00:00:00,34.9900
3,708,2011-05-31 00:00:00,2012-05-29 00:00:00,33.6442,2012-05-29 00:00:00,33.6442
4,708,2012-05-30 00:00:00,2013-05-29 00:00:00,33.6442,2013-05-29 00:00:00,33.6442
...,...,...,...,...,...,...
390,995,2013-05-30 00:00:00,None,101.2400,2013-05-09 00:00:00,101.2400
391,996,2013-05-30 00:00:00,None,121.4900,2013-05-09 00:00:00,121.4900
392,997,2013-05-30 00:00:00,None,539.9900,2013-05-09 00:00:00,539.9900
393,998,2013-05-30 00:00:00,None,539.9900,2013-05-09 00:00:00,539.9900


### TransactionHistory

In [21]:
#Collecting tables
transactionHistory = pd.read_sql('SELECT * FROM "Production.TransactionHistory"', adventureworks_sqlite_conn)
transactionHistoryArchive = pd.read_sql('SELECT * FROM "Production.TransactionHistoryArchive"', adventureworks_sqlite_conn)

#Merges
transactionHistory = pd.merge(transactionHistory, transactionHistoryArchive, on=['TransactionID'], how='outer', suffixes=('', '_transactionHistoryArchive'))

#Get most recent ModifiedDate
transactionHistory['ModifiedDate'] = transactionHistory.apply(lambda row: row['ModifiedDate_transactionHistoryArchive'] if pd.to_datetime(row['ModifiedDate_transactionHistoryArchive']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
transactionHistory.drop(columns=['ModifiedDate_transactionHistoryArchive'], inplace=True)

transactionHistory

,TransactionID,ProductID,ReferenceOrderID,ReferenceOrderLineID,TransactionDate,TransactionType,Quantity,ActualCost,ModifiedDate,ProductID_transactionHistoryArchive,ReferenceOrderID_transactionHistoryArchive,ReferenceOrderLineID_transactionHistoryArchive,TransactionDate_transactionHistoryArchive,TransactionType_transactionHistoryArchive,Quantity_transactionHistoryArchive,ActualCost_transactionHistoryArchive
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2011-04-16 00:00:00,P,4.0,50.2600
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.0,2.0,1.0,2011-04-16 00:00:00,P,3.0,45.1200
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,2.0,2.0,2011-04-16 00:00:00,P,3.0,45.5805
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530.0,3.0,1.0,2011-04-16 00:00:00,P,550.0,16.0860
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,1.0,2011-04-16 00:00:00,P,3.0,57.0255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202691,213438,332.0,3998.0,1.0,2014-08-03 00:00:00,P,550.0,10.8150,2014-08-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202692,213439,907.0,3999.0,2.0,2014-08-03 00:00:00,P,550.0,82.8345,2014-08-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202693,213440,948.0,3999.0,1.0,2014-08-03 00:00:00,P,550.0,82.8345,2014-08-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202694,213441,325.0,4000.0,1.0,2014-08-03 00:00:00,P,1250.0,0.2100,2014-08-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ProductReview

In [22]:
#Collecting tables
productReview = pd.read_sql('SELECT * FROM "Production.ProductReview"', adventureworks_sqlite_conn)

productReview

,ProductReviewID,ProductID,ReviewerName,ReviewDate,EmailAddress,Rating,Comments,ModifiedDate
0,1,709,John Smith,2013-09-18 00:00:00,john@fourthcoffee.com,5,I can't believe I'm singing the praises of a p...,2013-09-18 00:00:00
1,2,937,David,2013-11-13 00:00:00,david@graphicdesigninstitute.com,4,"A little on the heavy side, but overall the en...",2013-11-13 00:00:00
2,3,937,Jill,2013-11-15 00:00:00,jill@margiestravel.com,2,Maybe it's just because I'm new to mountain bi...,2013-11-15 00:00:00
3,4,798,Laura Norman,2013-11-15 00:00:00,laura@treyresearch.net,5,The Road-550-W from Adventure Works Cycles is ...,2013-11-15 00:00:00


### EmployeeDepartmentHistory

In [23]:
#Collecting tables
employeeDepartmentHistory = pd.read_sql('SELECT * FROM "HumanResources.EmployeeDepartmentHistory"', adventureworks_sqlite_conn)

employeeDepartmentHistory

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13 00:00:00
1,2,1,1,2008-01-31,None,2008-01-30 00:00:00
2,3,1,1,2007-11-11,None,2007-11-10 00:00:00
3,4,1,1,2007-12-05,2010-05-30,2010-05-28 00:00:00
4,4,2,1,2010-05-31,None,2010-05-30 00:00:00
...,...,...,...,...,...,...
291,286,3,1,2013-05-30,None,2013-05-29 00:00:00
292,287,3,1,2012-04-16,None,2012-04-15 00:00:00
293,288,3,1,2013-05-30,None,2013-05-29 00:00:00
294,289,3,1,2012-05-30,None,2012-05-29 00:00:00


### EmployeePayHistory

In [24]:
#Collecting tables
employeePayHistory = pd.read_sql('SELECT * FROM "HumanResources.EmployeePayHistory"', adventureworks_sqlite_conn)

employeePayHistory

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate
0,1,2009-01-14 00:00:00,125.5000,2,2014-06-30 00:00:00
1,2,2008-01-31 00:00:00,63.4615,2,2014-06-30 00:00:00
2,3,2007-11-11 00:00:00,43.2692,2,2014-06-30 00:00:00
3,4,2007-12-05 00:00:00,8.6200,2,2007-11-21 00:00:00
4,4,2010-05-31 00:00:00,23.7200,2,2010-05-16 00:00:00
...,...,...,...,...,...
311,286,2013-05-30 00:00:00,23.0769,2,2014-06-30 00:00:00
312,287,2012-04-16 00:00:00,48.1010,2,2014-06-30 00:00:00
313,288,2013-05-30 00:00:00,23.0769,2,2014-06-30 00:00:00
314,289,2012-05-30 00:00:00,23.0769,2,2014-06-30 00:00:00


# Convert Dataframes into Sqlite Tables 

In [41]:
conn = sqlite3.connect('../data/processed/United_Outdoors.sqlite3')

# Function to handle duplicate column names
def handle_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.str.lower().duplicated()].unique(): 
        mask = cols.str.lower() == dup.lower()
        cols.loc[mask] = cols.loc[mask] + '_' + cols.loc[mask].groupby(cols).cumcount().astype(str)
        cols.loc[mask & (cols == dup.lower() + '_0')] = dup
    df.columns = cols
    return df

import re

def get_sqlite_type(col, dtype):
    if re.search(r'id.*\d', col.lower()):
        return 'INTEGER'
    elif pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    elif pd.api.types.is_object_dtype(dtype):
        return 'VARCHAR(255)'
    else:
        return 'TEXT'

def write_df_to_sqlite(df, table_name, conn):
    # Handle duplicate columns
    df = handle_duplicate_columns(df)

    # If 'id' column doesn't exist, create it
    if 'id' not in df.columns:
        df = df.reset_index().rename(columns={'index': 'id'})
    else:
        # Generate unique 'id' values for duplicate rows
        df['id'] = df.groupby(df.columns.tolist()).ngroup() + 1

    # Drop the table if it already exists
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Create the table with 'id' as the primary key and other columns with their original data types
    conn.execute(f"""
        CREATE TABLE {table_name} (
            id INTEGER PRIMARY KEY,
            {', '.join(f'{col} {get_sqlite_type(col, df[col].dtype)}' for col in df.columns if col != 'id')}
        )
    """)

    # Write the DataFrame to the SQLite database
    df.to_sql(table_name, conn, if_exists='append', index=False)

def write_product_to_sqlite(df, table_name, conn):
    # Handle duplicate columns
    df = handle_duplicate_columns(df)

    # If 'id' column doesn't exist, create it
    if 'id' not in df.columns:
        df.insert(0, 'id', range(1, len(df) + 1))
    else:
        # Generate unique 'id' values for duplicate rows
        df['id'] = df.groupby(df.columns.tolist()).ngroup() + 1

    # Drop the table if it already exists
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Create the table with 'id' as the primary key
    conn.execute(f"""
        CREATE TABLE {table_name} (
            id INTEGER PRIMARY KEY,
            {', '.join(f'{col} {get_sqlite_type(col, df[col].dtype)}' for col in df.columns if col != 'id')}
        )
    """)

    # Write the DataFrame to the SQLite database
    df.to_sql(table_name, conn, if_exists='append', index=False)

def write_stateProvince_to_sqlite2(df, table_name, conn):
    # Handle duplicate columns
    df = handle_duplicate_columns(df)

    # If 'id' column doesn't exist, create it
    if 'id' not in df.columns:
        df.insert(0, 'id', range(1, len(df) + 1))
    else:
        # Ensure 'id' column has unique values
        df['id'] = range(1, len(df) + 1)

    # Drop the table if it already exists
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Create the table with 'id' as the primary key
    conn.execute(f"""
        CREATE TABLE {table_name} (
            id INTEGER PRIMARY KEY,
            {', '.join(f'{col} TEXT' for col in df.columns if col != 'id')}
        )
    """)

    # Write the DataFrame to the SQLite database
    df.to_sql(table_name, conn, if_exists='append', index=False)

def write_customer_to_sqlite(df, table_name, conn):
    # Handle duplicate columns
    df = handle_duplicate_columns(df)

    # If 'id' column doesn't exist, create it
    if 'id' not in df.columns:
        df = df.reset_index().rename(columns={'index': 'id'})

    # Drop the table if it already exists
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Create the table with 'id' as the primary key
    conn.execute(f"""
        CREATE TABLE {table_name} (
            id INTEGER PRIMARY KEY,
            {', '.join(f'{col} TEXT' for col in df.columns if col != 'id')}
        )
    """)

    # Write the DataFrame to the SQLite database
    df.to_sql(table_name, conn, if_exists='append', index=False)

In [42]:
write_df_to_sqlite(person, 'Person', conn)
write_product_to_sqlite(product, 'Product', conn)
write_product_to_sqlite(employee, 'Employee', conn)
write_product_to_sqlite(customer, 'Customer', conn)
write_product_to_sqlite(stateProvince, 'StateProvince', conn)
write_df_to_sqlite(address, 'Address', conn)
write_df_to_sqlite(workOrderRouting, 'WorkOrderRouting', conn)
write_df_to_sqlite(workOrder, 'WorkOrder', conn)
write_df_to_sqlite(document, 'Document', conn)
write_df_to_sqlite(sales_order_item, 'SalesOrderItem', conn)
write_df_to_sqlite(bonus, 'Bonus', conn)
write_df_to_sqlite(purchaseOrderDetails, 'PurchaseOrderDetails', conn)
write_df_to_sqlite(productVendor, 'ProductVendor', conn)
write_df_to_sqlite(billOfMaterials, 'BillOfMaterials', conn)
write_df_to_sqlite(productListPriceHistory, 'ProductListPriceHistory', conn)
write_df_to_sqlite(transactionHistory, 'TransactionHistory', conn)
write_df_to_sqlite(productReview, 'ProductReview', conn)
write_df_to_sqlite(employeeDepartmentHistory, 'EmployeeDepartmentHistory', conn)
write_df_to_sqlite(employeePayHistory, 'EmployeePayHistory', conn)



stateProvince.columns

### Connecties sluiten

In [32]:
northwind_sqlite_conn.close()
adventureworks_sqlite_conn.close()
aenc_sqlite_conn.close()